# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import time
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import pickle

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import make_multilabel_classification
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///db_disaster_messages.db')
df = pd.read_sql_table('disaster_massages_table', engine) 

X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

    return clean_tokens

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
def model_pipeline():
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))    
    ])
    
    parameters = {
        #'clf__estimator__min_samples_split': [2, 4],
        #'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'text_pipeline__tfidf__use_idf': (True, False),

    }   
    
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=4, n_jobs=-1)
    
    return pipeline

def model_pipeline_boosted():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

def model_pipeline_boosted_2():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'clf__estimator__min_samples_split': [2, 4],
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'text_pipeline__tfidf__use_idf': (True, False),
    }   
    
    cv = GridSearchCV(pipeline, param_grid=parameters)  
    
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
def main():

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = model_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    overall_accuracy = (y_pred == y_test).mean().mean()*100
    print('Model overall Accuracy: ', overall_accuracy, '\n')
    
    display_results(model, y_test, y_pred)    

In [7]:
def main_boosted():

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = model_pipeline_boosted()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    overall_accuracy = (y_pred == y_test).mean().mean()*100
    print('Model overall Accuracy: ', overall_accuracy, '\n')

    display_results(model, y_test, y_pred)

In [8]:
def main_boosted_2():

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = model_pipeline_boosted_2()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    overall_accuracy = (y_pred == y_test).mean().mean()*100
    print('Model overall Accuracy: ', overall_accuracy, '\n')

    display_results(model, y_test, y_pred)

In [9]:
def display_results(cv, y_test, y_pred):

    y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)

    for column in y_test.columns:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column], y_pred_df[column]))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
start_time = time.time()
main()
print("--- %s seconds ---" % (time.time() - start_time))

Model overall Accuracy:  94.2571118571 

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

          0       0.61      0.35      0.44      1560
          1       0.81      0.93      0.87      4946
          2       0.17      0.02      0.04        48

avg / total       0.76      0.79      0.76      6554

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5448
          1       0.80      0.39      0.53      1106

avg / total       0.87      0.88      0.86      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

-----------------------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
#start_time = time.time()
#main_boosted()
#print("--- %s seconds ---" % (time.time() - start_time))

Model overall Accuracy:  94.15200217 

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

          0       0.60      0.33      0.43      1501
          1       0.82      0.93      0.87      5010
          2       0.56      0.12      0.19        43

avg / total       0.77      0.79      0.77      6554

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5451
          1       0.82      0.37      0.51      1103

avg / total       0.87      0.88      0.86      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6518
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6554

-------------------------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
pipe = pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pipe.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [42]:
start_time = time.time()
main_boosted_2()
print("--- %s seconds ---" % (time.time() - start_time))

Model overall Accuracy:  94.3261960465 

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

          0       0.62      0.34      0.44      1586
          1       0.81      0.93      0.87      4924
          2       0.42      0.23      0.29        44

avg / total       0.76      0.78      0.76      6554

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5461
          1       0.83      0.38      0.52      1093

avg / total       0.88      0.88      0.86      6554

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

-----------------------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [55]:
pickle.dump(main, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Summary:

### Main():
    - Accurancy: 94.2736
    - Executation time: 76.71 seconds

### Main_boosted():
    - Accurancy: 94.1286
    - Executation time: 135.15 seconds
    
### Main_boosted_2():
    - Accurancy: 94.3261
    - Executation time: 873.03 seconds
    
There are definily a few ways to increase the pipeline accurancy using better parameters, however it's necessary more Computational processing resources. Since the simples approach (Main) had a good accurancy and a good time execution, I decided to follow with the most basic one for this project